# X-ray diffraction (XRD) interactive plot

*Author: Enze Chen, PhD student at the University of California, Berkeley*

This is an interactive notebook for playing around with some experimental parameters ($a$, $\lambda$, $T$, etc.) and observing the effect on the resulting XRD spectra. I find XRD to be a particularly beautiful subject and I couldn't find any similar visualizations online. I hope this interactive demo will help you learn the general _qualitative trends_ associated with powder XRD spectra. Please don't hesitate to reach out if you have questions or ideas to contribute.

## Acknowledgements

I would like to thank Prof. Andrew Minor for teaching MATSCI 204: Materials Characterization and my advisor Prof. Mark Asta for his unwavering encouragement. Interactivity is enabled with the [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/) library.

## Important equations

The most important equation is Bragg's law, given by 

$$n\lambda = 2d \sin(\theta)$$

where $n$ is the order (typically $1$), $\lambda$ is the wavelength, $d$ is the interplanar spacing, and $\theta$ is the Bragg angle. In creating the interactive widget, we will need to solve for $\theta$ as follows:

$$ \lambda = 2d \sin(\theta) \longrightarrow \theta = \sin^{-1} \left( \frac{\lambda}{2d} \right), \quad d = \frac{a}{\sqrt{h^2 + k^2 + l^2}} $$

where $h,k,l$ are the miller indices of the diffracting plane and $a$ is the lattice constant. 

Another important equation is the intensity, given by

$$ I = |F|^2 \times P \times m \times L \times A \times T $$

where
* $F$ is the Structure factor.
* $P$ is the Polarization factor.
* $m$ is the Multiplicity factor.
* $L$ is the Lorentz factor.
* $A$ is the Absorption factor.
* $T$ is the Temperature factor.

Furthermore, size effects can be included through the Scherrer equation, given by 

$$ t = \frac{K\lambda}{\beta \cos(\theta)} $$ 

where $t$ is the thickness, $K \sim 0.9$, and $\beta$ is the FWHM of the peak in radians.

For more information, please reference [Elements of X-Ray Diffraction (3rd) - Cullity and Stock](https://www.pearson.com/us/higher-education/program/Cullity-Elements-of-X-Ray-Diffraction-3rd-Edition/PGM113710.html).

### Assumptions I've taken great liberties with

* For the structure factor, I greatly oversimplified the construction of the atomic scattering factor and selected some numbers that more or less came from the data for iron since it has both BCC and FCC structures.
* I also combined part of the Temperature factor into the structure factor.
* I combined the Lorentz and Polarization factors, as is commonly done in the literature.
* Like any good computationalist (jk), I threw out most constant factors.
* I ignored the absorption factor since it is more or less independent of $\theta$.
* I used a $\sqrt[3]{\theta}$ term to approximate the thermal background's general shape. I don't know the actual dependence, if there is one.
* It only models FCC and BCC structures with single-atom bases. I think it's enough to get the trends, as calculations for specific materials can be found online (e.g. at [The Materials Project](https://materialsproject.org/)).
* I used a Gaussian distribution to model each peak so that I could capture crystallite size effects using the Scherrer equation. Peaks in general are not Gaussian.

### Known issues

* It doesn't have great safeguards against numerical errors, such as invalid `arcsin` arguments (e.g. if $\lambda$ is large and $d$ is small). Please be gentle. ❤
* There's a weird rendering error where for large intensities the upper limit (`1e6` for me) appears on the y-axis. **shrug**

## Python package imports

These are all the required Python packages.

In [1]:
# General libraries
import itertools

# Scientific computing libraries
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

# Interactivity libraries
from ipywidgets import interact_manual, fixed, \
                       IntSlider, FloatSlider, FloatLogSlider, RadioButtons, \
                       Button, Layout

## Widget function

This function handles all the calculations.

In [2]:
def plot_XRD(a, wavelength, cell_type, thickness, T=0, K=0.94):
    
    # Crystallographic planes
    planes = [[1,0,0], [1,1,0], [1,1,1], [2,0,0], [2,1,0], [2,1,1], [2,2,0],\
              [2,2,1], [3,0,0], [3,1,0], [3,1,1], [2,2,2], [3,2,0], [3,2,1]]
    planes_str = ['(100)', '(110)', '(111)', '(200)', '(210)', '(211)', '(220)',\
                  '(221)', '(300)', '(310)', '(311)', '(222)', '(320)', '(321)']

    # Set the basis
    basis = []
    if cell_type is 'FCC':
        basis = np.array([[0,0,0], [0.5,0.5,0], [0.5,0,0.5], [0,0.5,0.5]])
    elif cell_type is 'BCC':
        basis = np.array([[0,0,0], [0.5,0.5,0.5]])
    else:
        raise ValueError('Cell type not yet supported.')

    # Convert planes to theta values (see equation above)
    a += 1e-5 * T  # thermal expansion estimate
    s_vals = np.array([np.linalg.norm(p) for p in planes])
    theta = np.arcsin(np.divide(wavelength/2, np.divide(a, s_vals)))
    two_theta = 2 * np.degrees(theta)

    # Scherrer equation calculations
    beta = np.degrees(K * wavelength / thickness * np.divide(1, np.cos(theta)))
    sigma = beta / 2.355  # proportionality for Gaussian distribution

    # Structure-Temperature factor. Must... resist... for loops...
    s = np.sin(theta) / wavelength
    f = 26 - 41.8 * 7 * np.multiply(s**2, np.exp(-(s)**2))
    F = np.multiply(f, np.sum(np.exp(2 * np.pi * 1j * \
                                     np.dot(np.array(planes), basis.T)), axis=1))

    # Multiplicity factor
    mult = [2**np.count_nonzero(p) * \
            sum(1 for _ in set(itertools.permutations(p))) for p in planes]

    # Lorentz-Polarization factor
    P = np.divide(1 + np.cos(2 * theta)**2, np.multiply(np.sin(theta)**2, np.cos(theta)))

    # Final intensity (numpy hacks)
    factors = [np.absolute(F)**2, mult, P]
    I = np.prod(np.vstack(factors), axis=0)
    
    # Plotting
    plt.rcParams.update({'figure.figsize':(10,5), 'font.size':22})
    xmin, xmax = (20, 160)
    fig, ax = plt.subplots()

    # Create x-axis points
    x = np.linspace(xmin, xmax, int(5e4))
    thermal_diffuse = 1e2 * T * np.cbrt(x)  # THIS FUNCTIONAL DEPENDENCE IS NOT REAL!!!
                                            # Chosen just to get general shape of background
    
    # Save all the curves, then take a max envelope
    all_curves = []
    for i in range(len(sigma)):
        y = stats.norm.pdf(x, two_theta[i], sigma[i])
        normed_curve = y / max(y) * I[i]
        # Don't include the curves that aren't selected by the Structure factor
        if max(normed_curve) > 1e1:
            max_ind = normed_curve.argmax()
            ax.annotate(s=planes_str[i], \
                        xy=(x[max_ind], normed_curve[max_ind] + thermal_diffuse[max_ind]))
            all_curves.append(normed_curve)
    final_curve = np.max(all_curves, axis=0) + thermal_diffuse
    plt.plot(x, final_curve, c='C0', lw=4)

    # Some fine-tuned settings for visual appeal
    for side in ['top', 'right']:
        ax.spines[side].set_visible(False)
    for side in ['left', 'bottom']:
        ax.spines[side].set_linewidth(2)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(0, 1.05*ax.get_ylim()[1])
    ax.tick_params(left=False, labelleft=False, direction='in', length=10, width=2)
    ax.set_xlabel(r'$2\theta\ (^{\circ})$')
    ax.set_ylabel('Intensity (a.u.)')
    plt.show()

I've created each slider individually for readability and customization.

In [3]:
b = Button(description='A button', layout=Layout(width='400px', height='40px'))

a_widget = FloatSlider(value=0.352, min=0.25, max=0.45, step=0.001, description='Lattice constant (nm)', \
                       readout_format='.3f', style={'description_width':'150px'}, layout=b.layout)

w_widget = FloatSlider(value=0.154, min=0.070, max=0.200, step=0.001, description='X-ray wavelength (nm)',\
                       readout_format='.3f', style={'description_width':'150px'}, layout=b.layout)

c_widget = RadioButtons(options=['FCC', 'BCC'], description='Crystal structure',\
                        style={'description_width':'150px'}, layout=b.layout)

t_widget = FloatLogSlider(value=10, base=10, min=0, max=3, step=0.1, description='Crystallite size (nm)',\
                          readout_format='d', style={'description_width':'150px'}, layout=b.layout)

T_widget = IntSlider(value=298, min=0, max=1000, step=1, description='Temperature (K)',\
                     readout_format='d', style={'description_width':'150px'}, layout=b.layout)

And the final touch.

In [4]:
interact_manual.opts['manual_name'] = 'Generate plot'
interact_manual(plot_XRD, a=a_widget, wavelength=w_widget, cell_type=c_widget, 
                          thickness=t_widget, T=T_widget, K=fixed(0.94));

interactive(children=(FloatSlider(value=0.352, description='Lattice constant (nm)', layout=Layout(height='40px…